In [1]:
# %% [markdown]
# # Bottom (Lowest) 10 Products - Analysis
# Identify lowest 10 sellers (value + quantity) over last 12 months, build metrics.

import pandas as pd
import numpy as np
from pathlib import Path

# Config
DATA_PATH = Path(r"E:\c drive\amazon\data\online_retail\online_retail_II.xlsx")
OUT_DIR = Path("./outputs_low_sellers")
OUT_DIR.mkdir(parents=True, exist_ok=True)
ROLLING_MONTHS = 12

# Load & clean
df = pd.read_excel(DATA_PATH, parse_dates=['InvoiceDate'])
df = df.dropna(subset=['Description', 'Quantity', 'Price', 'InvoiceDate'])
df = df[(df['Quantity'] > 0) & (df['Price'] > 0)]
df['Description'] = df['Description'].str.strip()
df['SalesValue'] = df['Quantity'] * df['Price']

# Filter last 12 months (if applicable)
max_date = df['InvoiceDate'].max()
cutoff = max_date - pd.DateOffset(months=ROLLING_MONTHS)
df_12m = df[df['InvoiceDate'] >= cutoff].copy()
if df_12m.empty:
    df_12m = df.copy()

# Aggregate
agg = (
    df_12m.groupby('Description')
    .agg(Total_Value=('SalesValue','sum'),
         Total_Quantity=('Quantity','sum'),
         Orders=('Invoice','nunique'),
         Avg_Price=('Price','mean'))
    .reset_index()
)

# Exclude service-like items (optional)
exclude_keywords = ['POSTAGE', 'DOTCOM', 'Manual']
mask_exclude = agg['Description'].str.upper().str.contains('|'.join(exclude_keywords))
agg_filtered = agg[~mask_exclude].copy()

# Bottom 10 by Total_Value
bottom10_value = agg_filtered.sort_values('Total_Value', ascending=True).head(10)
# Bottom 10 by Quantity
bottom10_qty = agg_filtered.sort_values('Total_Quantity', ascending=True).head(10)

# Union
bottom_set = sorted(set(bottom10_value['Description']).union(set(bottom10_qty['Description'])))
bottom_union = pd.DataFrame(bottom_set, columns=['Description'])
bottom_union.to_csv(OUT_DIR / "bottom10_products.csv", index=False)

print("Bottom unique product count:", len(bottom_union))
display(bottom_union)

# Monthly time series for these products
df_12m['YearMonth'] = df_12m['InvoiceDate'].dt.to_period('M').dt.to_timestamp()
monthly = (
    df_12m[df_12m['Description'].isin(bottom_union['Description'])]
    .groupby(['YearMonth','Description'])
    .agg(Month_Value=('SalesValue','sum'),
         Month_Quantity=('Quantity','sum'),
         Orders=('Invoice','nunique'))
    .reset_index()
)

monthly.to_csv(OUT_DIR / "bottom10_products_monthly.csv", index=False)
display(monthly.head())

# Feature engineering for discount model
feat = (
    monthly.groupby('Description')
    .agg(
        Avg_Monthly_Quantity=('Month_Quantity','mean'),
        Avg_Monthly_Value=('Month_Value','mean'),
        Months_Active=('Month_Quantity','count'),
        Total_Quantity=('Month_Quantity','sum'),
        Total_Value=('Month_Value','sum'),
        Std_Monthly_Quantity=('Month_Quantity','std')
    )
    .reset_index()
)

# Zero-sales months: we need a consistent calendar grid (if some months missing)
all_months = pd.date_range(monthly['YearMonth'].min(), monthly['YearMonth'].max(), freq='MS')
products = bottom_union['Description'].tolist()
grid = pd.MultiIndex.from_product([all_months, products], names=['YearMonth','Description'])
full = monthly.set_index(['YearMonth','Description']).reindex(grid, fill_value=0).reset_index()

zero_ratio = (
    full.groupby('Description')['Month_Quantity']
    .apply(lambda x: (x==0).mean())
    .reset_index(name='Zero_Sales_Ratio')
)

# Last sale recency in months
last_sale = (
    full[full['Month_Quantity']>0]
    .groupby('Description')['YearMonth']
    .max()
    .reset_index()
    .rename(columns={'YearMonth':'Last_Sale_Date'})
)
last_sale['Last_Sale_Months_Ago'] = ((max_date.to_period('M').to_timestamp() - last_sale['Last_Sale_Date'])/np.timedelta64(1,'M'))

feat = feat.merge(zero_ratio, on='Description', how='left')
feat = feat.merge(last_sale[['Description','Last_Sale_Months_Ago']], on='Description', how='left')

feat['CV_Monthly_Quantity'] = feat['Std_Monthly_Quantity'] / feat['Avg_Monthly_Quantity'].replace(0,np.nan)

feat.to_csv(OUT_DIR / "bottom10_features.csv", index=False)
display(feat)

print("Files written to:", OUT_DIR.resolve())
for f in OUT_DIR.iterdir():
    print(" -", f.name)


Bottom unique product count: 20


,Description
0,*Boombox Ipod Classic
1,"BISCUIT TIN, MINT,IVORY, VINTAGE"
2,BLUE WHITE PLASTIC RINGS LAMPSHADE
3,CAT W SUNGLASSES BLANK CARD
4,CHAMPAGNE TRAY BLANK CARD
5,HAPPY BIRTHDAY CARD TEDDY/CAKE
6,HAPPY BIRTHDAY GINGER CAT CARD
7,PADS TO MATCH ALL CUSHIONS
8,PINK HEART CHRISTMAS DECORATION
9,PURPLE CRYSTAL DROP EARRINGS


,YearMonth,Description,Month_Value,Month_Quantity,Orders
0,2009-12-01,PINK HEART CHRISTMAS DECORATION,0.380,1,1
1,2009-12-01,RAIN GIRL CHILDS UMBRELLA,3.250,1,1
2,2010-01-01,PADS TO MATCH ALL CUSHIONS,0.002,2,2
3,2010-01-01,RED HEART CANDY POP LIGHTS,4.950,1,1
4,2010-01-01,RUSSIAN FOLKART STACKING TINS,4.950,1,1


ValueError: Unit M is not supported. Only unambiguous timedelta values durations are supported. Allowed units are 'W', 'D', 'h', 'm', 's', 'ms', 'us', 'ns'